## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-21-53-17 +0000,bbc,"Peers trying to block assisted dying, claims M...",https://www.bbc.com/news/articles/c5yqdpxzzxlo...
1,2025-11-21-21-49-25 +0000,nyt,RFK Jr. Says He Instructed CDC to Change Vacci...,https://www.nytimes.com/2025/11/21/us/politics...
2,2025-11-21-21-48-02 +0000,cbc,Top court says federal prisoners can challenge...,https://www.cbc.ca/news/politics/supreme-court...
3,2025-11-21-21-47-29 +0000,nypost,On-duty NYPD school safety agent busted for gr...,https://nypost.com/2025/11/21/us-news/on-duty-...
4,2025-11-21-21-46-14 +0000,nypost,"Mamdani, Trump’s expected slugfest turns into ...",https://nypost.com/2025/11/21/us-news/mamdani-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,46
345,new,26
65,ukraine,17
68,plan,16
67,peace,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-11-21-20-01-53 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...,118
10,2025-11-21-21-33-52 +0000,nypost,Trump says Zohran Mamdani is not a ‘jihadist’ ...,https://nypost.com/2025/11/21/us-news/trump-br...,114
38,2025-11-21-20-24-09 +0000,nyt,"As Trump Pushes on Russia-Ukraine Peace Plan, ...",https://www.nytimes.com/2025/11/21/world/europ...,113
93,2025-11-21-18-01-09 +0000,bbc,Trump says he expects 'civil' meeting at White...,https://www.bbc.com/news/articles/cvgmgddyjl0o...,111
372,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,109


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,118,2025-11-21-20-01-53 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...
340,84,2025-11-20-23-31-36 +0000,nypost,Ex-New York Post media columnist Keith J. Kell...,https://nypost.com/2025/11/20/media/ex-new-yor...
10,68,2025-11-21-21-33-52 +0000,nypost,Trump says Zohran Mamdani is not a ‘jihadist’ ...,https://nypost.com/2025/11/21/us-news/trump-br...
358,56,2025-11-20-22-37-38 +0000,latimes,Driver charged with murder in crash that kille...,https://www.latimes.com/california/story/2025-...
309,55,2025-11-21-01-36-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
142,51,2025-11-21-15-53-00 +0000,wsj,The Democratic National Committee took out a $...,https://www.wsj.com/politics/policy/democratic...
151,43,2025-11-21-15-12-08 +0000,nypost,NY Fed president floats chance of a rate cut i...,https://nypost.com/2025/11/21/business/ny-feds...
182,39,2025-11-21-13-00-00 +0000,wsj,When Toyota was planning to build its first st...,https://www.wsj.com/arts-culture/books/martha-...
43,35,2025-11-21-20-17-11 +0000,bbc,Watch: Pope Leo to US students: Don't let AI d...,https://www.bbc.com/news/videos/c99704g0x4ro?a...
300,35,2025-11-21-02-31-01 +0000,wapo,U.S. Coast Guard will no longer classify swast...,https://www.washingtonpost.com/national-securi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
